In [ ]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP, GRABER
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PyQuanteDriver, UnitsType, BaseDriver
from qiskit.chemistry import FermionicOperator
import time

In [ ]:
def get_qubit_op(dist):
    driver = PyQuanteDriver(atoms="Li .0 .0 .0; H .0 .0 " + str(dist), units=UnitsType.ANGSTROM, 
                         charge=0)
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) #qubit tapering
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift

In [ ]:
start = time.time()
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 3.5, 0.1)
exact_energies = []
rel_exact_energies =[]
vqe_energies = []
rel_vqe_energies = []

optimizer = GRABER() #including random parameter choise
initial_point = None
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    rel_exact_energies.append(np.real(result.eigenvalues))
    exact_energies.append(np.real(result.eigenvalues + shift))
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer,initial_point = initial_point)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'])
    rel_vqe_energies.append(vqe_result)
    vqe_energies.append(vqe_result + shift)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    current = vqe.optimal_params
    initial_point = current
    
print("All energies have been calculated")
end = time.time() -start
print(end)

In [ ]:
plt.plot(distances, [i+1.2 for i in rel_exact_energies], 'o', label="Exact Energy")
plt.plot(distances, [i+1.2 for i in rel_vqe_energies], 'x', label="VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
plt.show()

In [ ]:
plt.plot(distances, exact_energies)
plt.plot(distances, vqe_energies)

In [ ]:
plt.plot(distances, [abs(vqe_energies[i]-exact_energies[i]) for i in range(len(vqe_energies))], label = 'Rel Error')
plt.legend();

In [ ]:
from scipy import optimize

#V (r) = De(1 − e−α(r−r0))^2
def test_func(x, a, b, c, d):
    return a * ((1- np.exp(-b*(x-c)))**2) + d

params, params_covariance = optimize.curve_fit(test_func, distances, vqe_energies, p0=[1, -1, -1.9, -7.9])

print(params)
print(params[0], "*((1-np.exp(-", params[1], " * (x-", params[2], ")))**2) + ",params[3])

fitted_energies = []
for i in range(len(exact_energies)):
#     print(test_func(distances[i], params[0], params[1], params[2]))
    fitted_energies.append(test_func(distances[i], params[0], params[1], params[2], params[3]))
    
#plt.plot(distances, exact_energies, 'o', label="Exact Energy")
plt.plot(distances, vqe_energies, 'x', label="VQE Energy")
plt.plot(distances, fitted_energies,label='Fitted function')
plt.plot(distances, exact_energies, '--', label = "Exact Energies")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend(loc='best')
plt.show()